In [1]:
clsModelsDict = ({
    "meat": "models/classification_models/YOLO/meat.pt",
    "vegetable": "models/classification_models/YOLO/vegetable.pt",
    "fruit": "models/classification_models/YOLO/fruit.pt",
    "cheese-dairy" : "models/classification_models/YOLO/cheese-dairy.pt",
    "bread-pasta-grains": "models/classification_models/YOLO/bread-pasta-grains.pt",
    "nuts-seeds": "models/classification_models/YOLO/nuts-seeds.pt",
    "misc": "models/classification_models/YOLO/misc.pt",
})

In [2]:
import importlib
import FoodDetection

importlib.reload(FoodDetection)

from FoodDetection import FoodDetectionModel

app_model = FoodDetectionModel(
    detection_model_path="models/detection_models/detection.pt",
    classification_config=clsModelsDict,
    detection_id_to_name="dicts/detect_classes_v4.json",
    det_to_cls_group="dicts/det_to_cls_groups.json"
)

In [3]:
results = app_model.run("../model/data/test_pics/Salmon-Bowl-SQ-2.jpg", conf_threshold=0.3, det_imgsz=1800, verbose=True)
print(results)

WARNING imgsz=[1800] must be multiple of max stride 32, updating to [1824]
Detection 'carrot' mapped to classification group 'vegetable'
Detection 'cucumber' mapped to classification group 'vegetable'
Detection 'cucumber' mapped to classification group 'vegetable'
Detection 'rice' mapped to classification group 'bread-pasta-grains'
Detection 'chicken' mapped to classification group 'meat'
[{'pred_class_name': 'carrot', 'pred_class_id': 21, 'bbox': [481, 760, 880, 1191], 'det_class_name': 'carrot', 'det_conf_score': 0.7969042658805847, 'cls_group': 'vegetable', 'top5_cls_results': [{'class_name': 'carrot', 'probability': 0.895331621170044}, {'class_name': 'potato', 'probability': 0.10436476767063141}, {'class_name': 'mixed-vegetables', 'probability': 7.83534487709403e-05}, {'class_name': 'red-radish', 'probability': 4.646185698220506e-05}, {'class_name': 'red-cabbage', 'probability': 3.641293369582854e-05}]}, {'pred_class_name': 'cucumber', 'pred_class_id': 31, 'bbox': [771, 221, 1195, 

In [4]:
for result in results:
    print(f"Predicted Class Name: {result['pred_class_name']}")
    print("Top3 classes to choose from:")
    for i in range(3):
        if i < len(result["top5_cls_results"]):
            cls_info = result["top5_cls_results"][i]
            print(f"  {i+1}. {cls_info['class_name']} (Probability: {cls_info['probability']:.4f})")
    print("-------------"*3)

Predicted Class Name: carrot
Top3 classes to choose from:
  1. carrot (Probability: 0.8953)
  2. potato (Probability: 0.1044)
  3. mixed-vegetables (Probability: 0.0001)
---------------------------------------
Predicted Class Name: cucumber
Top3 classes to choose from:
  1. cucumber (Probability: 1.0000)
  2. mixed-vegetables (Probability: 0.0000)
  3. broccoli (Probability: 0.0000)
---------------------------------------
Predicted Class Name: rice
Top3 classes to choose from:
  1. rice (Probability: 0.9998)
  2. pasta (Probability: 0.0002)
  3. quinoa (Probability: 0.0000)
---------------------------------------
Predicted Class Name: chicken
Top3 classes to choose from:
  1. chicken (Probability: 0.9134)
  2. beef (Probability: 0.0848)
  3. shrimp-prawn-large (Probability: 0.0011)
---------------------------------------


In [1]:
def process_model_output(output: list[dict]) -> list[dict] | None:
    try:
        processed_results = []
        for item in output:
            products_list = []
            i = 0
            for product in item["top5_cls_results"]:
                if i >= 3:
                    break
                products_list.append(
                    {"product": product["class_name"], "probability": product["probability"]}
                )
                i += 1

            processed_results.append(products_list)

        return processed_results
    except Exception as e:
        print(f"Error during process_model_output: {e}")

output = [{'pred_class_name': 'carrot', 'pred_class_id': 21, 'bbox': [481, 760, 880, 1191], 'det_class_name': 'carrot', 'det_conf_score': 0.7969042658805847, 'cls_group': 'vegetable', 'top5_cls_results': [{'class_name': 'carrot', 'probability': 0.895331621170044}, {'class_name': 'potato', 'probability': 0.10436476767063141}, {'class_name': 'mixed-vegetables', 'probability': 7.83534487709403e-05}, {'class_name': 'red-radish', 'probability': 4.646185698220506e-05}, {'class_name': 'red-cabbage', 'probability': 3.641293369582854e-05}]}, {'pred_class_name': 'cucumber', 'pred_class_id': 31, 'bbox': [771, 221, 1195, 611], 'det_class_name': 'cucumber', 'det_conf_score': 0.6672435998916626, 'cls_group': 'vegetable', 'top5_cls_results': [{'class_name': 'cucumber', 'probability': 0.9999711513519287}, {'class_name': 'mixed-vegetables', 'probability': 7.128524430299876e-06}, {'class_name': 'broccoli', 'probability': 4.546155196294421e-06}, {'class_name': 'kolhrabi', 'probability': 3.332810820211307e-06}, {'class_name': 'zucchini', 'probability': 3.31468572767335e-06}]}, {'pred_class_name': 'rice', 'pred_class_id': 78, 'bbox': [731, 536, 1169, 948], 'det_class_name': 'rice', 'det_conf_score': 0.4303257167339325, 'cls_group': 'bread-pasta-grains', 'top5_cls_results': [{'class_name': 'rice', 'probability': 0.9997513890266418}, {'class_name': 'pasta', 'probability': 0.00023718980082776397}, {'class_name': 'quinoa', 'probability': 8.463189260510262e-06}, {'class_name': 'bread-nut', 'probability': 1.1158530242028064e-06}, {'class_name': 'bread-whole-grain', 'probability': 8.354963370038604e-07}]}, {'pred_class_name': 'chicken', 'pred_class_id': 24, 'bbox': [525, 451, 771, 697], 'det_class_name': 'chicken', 'det_conf_score': 0.342996746301651, 'cls_group': 'meat', 'top5_cls_results': [{'class_name': 'chicken', 'probability': 0.913429856300354}, {'class_name': 'beef', 'probability': 0.08478688448667526}, {'class_name': 'shrimp-prawn-large', 'probability': 0.0010824088240042329}, {'class_name': 'sausage', 'probability': 0.0006484665791504085}, {'class_name': 'ham', 'probability': 2.1613539502141066e-05}]}]

In [2]:
process_model_output(output)

[[{'product': 'carrot', 'probability': 0.895331621170044},
  {'product': 'potato', 'probability': 0.10436476767063141},
  {'product': 'mixed-vegetables', 'probability': 7.83534487709403e-05}],
 [{'product': 'cucumber', 'probability': 0.9999711513519287},
  {'product': 'mixed-vegetables', 'probability': 7.128524430299876e-06},
  {'product': 'broccoli', 'probability': 4.546155196294421e-06}],
 [{'product': 'rice', 'probability': 0.9997513890266418},
  {'product': 'pasta', 'probability': 0.00023718980082776397},
  {'product': 'quinoa', 'probability': 8.463189260510262e-06}],
 [{'product': 'chicken', 'probability': 0.913429856300354},
  {'product': 'beef', 'probability': 0.08478688448667526},
  {'product': 'shrimp-prawn-large', 'probability': 0.0010824088240042329}]]